## Test model on metadata

In [12]:
# Import Pytorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

# Import matplotlib for visulization
import matplotlib.pyplot as plt

import cv2

import pandas as pd  # 資料分析套件
import numpy as np
import os
from time import time
import re

### var setting

In [13]:
model_path = "../model/best_model_resnet18_7.pth"

dataset_path = "../HAM10000/meta_img_data"

groundtruth_file = '../HAM10000/GroundTruth.csv'

device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
from pytorch_helper.CreateFileLableDict import CreateFileLableDict
filename_to_label_dict = CreateFileLableDict(groundtruth_file)

所有的值都在範圍內：


### Create dataloader

In [15]:
import random
from torch.utils.data import DataLoader
from pytorch_helper.CustomImageDataset import CustomImageDataset

# 假設您的圖片和標籤字典如下
img_dir = dataset_path

# list all filename from file
all_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

# list all filename from dict
# all_files = list(filename_to_label_dict.keys())


num_total_samples = len(all_files)
split_ratio = 0.8  # 80% 的数据用于训练，20% 用于测试

# 随机打乱数据集
random.shuffle(all_files)

# 计算分割点
split_idx = int(num_total_samples * split_ratio)

# 分割数据集
train_files = all_files[:split_idx]
test_files = all_files[split_idx:]

transform224 = torchvision.transforms.Compose([
    transforms.Resize(size=(224,224)),
    torchvision.transforms.ToTensor()
    # 其他轉換
])


# 创建训练集和测试集的数据加载器
train_dataset = CustomImageDataset(img_dir=img_dir, file_to_label_dict={file: filename_to_label_dict[file] for file in train_files}, transform=transform224)
test_dataset = CustomImageDataset(img_dir=img_dir, file_to_label_dict={file: filename_to_label_dict[file] for file in test_files}, transform=transform224)
all_dataset = CustomImageDataset(img_dir=img_dir, file_to_label_dict={file: filename_to_label_dict[file] for file in all_files}, transform=transform224)

# 创建数据加载器
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
all_data_loader = DataLoader(all_dataset, batch_size=32, shuffle=False)


### Init training object

In [16]:
import torchvision.models as models
from pytorch_helper.helperFunctions import switchLastLayer
from pytorch_helper.TrainHelper import TrainingHelper

model_0 = models.resnet18(pretrained=True)     # 使用內建的 model 

# switchLastLayer(model_0, num_classes=7)

# change fully connected layer to 7
num_ftrs = model_0.fc.in_features
model_0.fc = nn.Linear(num_ftrs, 7)


model_0.load_state_dict(torch.load(model_path))

optimizer = torch.optim.SGD(model_0.parameters(), lr = 0.001) # 選擇你想用的 optimizer
# optimizer = torch.optim.Adam(model_VGG16.parameters(), lr =0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()                # 選擇想用的 loss functionmodel_vgg16 = 

model_0 = model_0.to(device)



trainingHelper = TrainingHelper(model=model_0,
                                train_dataloader=train_data_loader,
                                test_dataloader=all_data_loader,
                                loss_fn=loss_fn,
                                optimizer=optimizer,
                                device=device)

print(trainingHelper.test_step())

c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\E\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(0.5921973272206935, 0.8339370873074101, 0.8277992243345038)


### remove model's fc

In [17]:
from torchinfo import summary

model_0.to('cpu')

model_0.fc = torch.nn.Identity()

summary(model_0, input_size=[1,3,224,224])

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 512]                  --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│

In [18]:
# testing
from pytorch_helper.helperFunctions import img2tensor, flattenTensor

img_path = "../HAM10000/images/ISIC_0024306.jpg"

img = cv2.imread(img_path)
input_data = img2tensor(img).to(device)


output = model_0(input_data)

print(torch.softmax(output, dim=1))



# output = conv_layer(input_data)
# print(input_data.shape)
# print(output.shape)

# output = conv_block(input_data)
# output = conv_block_2(output)
# input_data = img2tensor(image_path)
# input_data = input_data.to(device)

# model_VGG16.eval()
# with torch.no_grad():
#     output = model_VGG16(input_data)
# prob = torch.softmax(output, dim=1)
# print(input_data.shape)
# print(output.shape)
# print(prob)
# print(flattenTensor(output).shape)

tensor([[0.0011, 0.0023, 0.0009, 0.0013, 0.0017, 0.0012, 0.0009, 0.0010, 0.0014,
         0.0018, 0.0019, 0.0011, 0.0018, 0.0009, 0.0012, 0.0009, 0.0010, 0.0014,
         0.0010, 0.0009, 0.0012, 0.0009, 0.0019, 0.0009, 0.0018, 0.0013, 0.0010,
         0.0016, 0.0030, 0.0012, 0.0010, 0.0016, 0.0022, 0.0011, 0.0009, 0.0009,
         0.0012, 0.0014, 0.0017, 0.0010, 0.0021, 0.0009, 0.0010, 0.0009, 0.0035,
         0.0023, 0.0119, 0.0009, 0.0040, 0.0009, 0.0014, 0.0009, 0.0011, 0.0009,
         0.0015, 0.0015, 0.0046, 0.0047, 0.0009, 0.0014, 0.0060, 0.0044, 0.0031,
         0.0011, 0.0009, 0.0010, 0.0028, 0.0042, 0.0089, 0.0010, 0.0037, 0.0019,
         0.0010, 0.0009, 0.0010, 0.0010, 0.0015, 0.0029, 0.0018, 0.0022, 0.0012,
         0.0014, 0.0041, 0.0020, 0.0011, 0.0015, 0.0011, 0.0040, 0.0016, 0.0010,
         0.0018, 0.0038, 0.0010, 0.0011, 0.0010, 0.0033, 0.0040, 0.0009, 0.0009,
         0.0010, 0.0024, 0.0052, 0.0009, 0.0009, 0.0015, 0.0013, 0.0017, 0.0026,
         0.0052, 0.0010, 0.0

### Generate CNN output data 

In [1]:
import os
import csv
import time
from pytorch_helper.helperFunctions import generateModelOutputs , saveCsv

input_folder = dataset_path
output_csvfile_name = "../resnet_output.csv"

model_0.to("cpu")

res = []

# open every img
start = time.time()
res = generateModelOutputs(model_0, dataset_path)
# for filename in os.listdir(input_folder):

#     image_path  = os.path.join(input_folder, filename)
#     # print(image_path)
#     img = cv2.imread(image_path)
#     img = img2tensor(img)

#     # model setting and eval
#     model_0.eval()
#     with torch.no_grad():
#         tensor = model_0(img)
    
#     tensor = flattenTensor(tensor)
#     vec = tensor.tolist()

#     vec = [filename] + vec
#     res.append(vec)

end = time.time()

print("time cost : {}".format(end - start))

# add header

# print(res)
with open(output_csvfile_name, 'w', newline='', encoding='utf-8') as csvfile:
    # try:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(res)
    # except Exception as e:
    #     logging.error("csv error : %s" , e)
    #     print('csv encoding error')


# add head

NameError: name 'dataset_path' is not defined